In [150]:
import pandas as pd
import warnings
warnings.filterwarnings(action="ignore")

## Lấy dữ liệu từ file

In [151]:
# Lấy số lượng stock, vị trí và thời gian nhập của từng mã hàng.
df_inventory=pd.read_excel(r"C:\Users\HP\Downloads\Python\python_picking\INPUT_inventory_01JAN.xlsx", parse_dates=['GR_time'])
# Sort theo SKU và GR (Good receipt) time
df_inventory=df_inventory.sort_values(by=['SKU','GR_time'],ascending=True).reset_index(drop=True)
# In ra màn hình
df_inventory.head()

,SKU,Stock_qty,UOM,Location,GR_time
0,BH208774,11,PCE,0321B,2020-05-30 16:03:00
1,BH208774,9,PCE,1419B,2020-07-16 09:55:00
2,BH208774,3,PCE,0304A,2020-07-16 09:55:00
3,BH208774,4,PCE,0302A,2020-07-22 13:52:00
4,BH208774,2,PCE,0304A,2020-07-24 16:42:00


In [152]:
# Lấy số lượng order của từng mã hàng
df_order=pd.read_excel(r"python_picking/INPUT_order_02JAN.xlsx",usecols=['SKU','Order_qty'])
# In ra màn hình
df_order.head()

,SKU,Order_qty
0,BH208774,27
1,BH20A130,157
2,BH20E001,75
3,BH209344A1,189
4,BY20A871,97


## Tính toán

### BƯỚC 1: GỘP FILE INVENTORY VÀ ORDER

In [153]:
# Merge (join) 2 file 'inventory.xlsx' và 'order.xlsx' làm 1 dựa trên SKU
# Để lấy cả thông tin inventory và số lượng order của từng mã hàng.
df_invt_order=df_order.merge(df_inventory, on='SKU')
# In ra màn hình
df_invt_order.head()

,SKU,Order_qty,Stock_qty,UOM,Location,GR_time
0,BH208774,27,11,PCE,0321B,2020-05-30 16:03:00
1,BH208774,27,9,PCE,1419B,2020-07-16 09:55:00
2,BH208774,27,3,PCE,0304A,2020-07-16 09:55:00
3,BH208774,27,4,PCE,0302A,2020-07-22 13:52:00
4,BH208774,27,2,PCE,0304A,2020-07-24 16:42:00


### BƯỚC 2: THEO TỪNG SKU - TÍNH LUỸ KẾ CHO SỐ LƯỢNG TỒN TRONG KHO

In [154]:
# Thêm cột số lượng luỹ kế
df_invt_order['SL_luy_ke']=df_invt_order.groupby('SKU')['Stock_qty'].apply(lambda x: x.cumsum())

# Thêm cột số lượng còn thiếu
df_invt_order['SL_con_thieu'] = df_invt_order['Order_qty'] - df_invt_order['SL_luy_ke'] 

# Thêm cột số dòng (sẽ giới thiệu mục đích sau) và set_index
df_invt_order['Row_num']=df_invt_order.index
df_invt_order.head(5)

,SKU,Order_qty,Stock_qty,UOM,Location,GR_time,SL_luy_ke,SL_con_thieu,Row_num
0,BH208774,27,11,PCE,0321B,2020-05-30 16:03:00,11,16,0
1,BH208774,27,9,PCE,1419B,2020-07-16 09:55:00,20,7,1
2,BH208774,27,3,PCE,0304A,2020-07-16 09:55:00,23,4,2
3,BH208774,27,4,PCE,0302A,2020-07-22 13:52:00,27,0,3
4,BH208774,27,2,PCE,0304A,2020-07-24 16:42:00,29,-2,4


### BƯỚC 3: FILTER, chỉ giữ lại các vị trí cuối cùng (không xuất hoặc chỉ xuất đi 1 phần)

In [155]:
# Thực hiện Filter
df_removed_before = df_invt_order.groupby('SKU').apply(lambda x:x[x['SL_con_thieu'] <= 0] )
df_removed_before = df_removed_before.reset_index(drop=True) #cột Row_num resets to the default integer index.
df_removed_before.head(10)

,SKU,Order_qty,Stock_qty,UOM,Location,GR_time,SL_luy_ke,SL_con_thieu,Row_num
0,BH208774,27,4,PCE,0302A,2020-07-22 13:52:00,27,0,3
1,BH208774,27,2,PCE,0304A,2020-07-24 16:42:00,29,-2,4
2,BH208774,27,16,PCE,0307A,2020-08-01 09:50:00,45,-18,5
3,BH208774,27,4,PCE,0307A,2020-08-01 09:50:00,49,-22,6
4,BH209344A1,189,14,PCE,0318A,2020-08-07 13:32:00,197,-8,33
5,BH209344A1,189,15,PCE,1217C,2020-08-08 14:13:00,212,-23,34
6,BH209344A1,189,78,PCE,0503A,2020-08-08 14:13:00,290,-101,35
7,BH209344A1,189,3,PCE,0328A,2020-08-08 17:54:00,293,-104,36
8,BH209344A1,189,81,PCE,0523A,2020-08-08 17:54:00,374,-185,37
9,BH209344A1,189,19,PCE,0132D,2020-08-08 17:54:00,393,-204,38


### BƯỚC 4: TÌM CÁC VỊ TRÍ MÀ TẠI ĐÓ PICK 1 PHẦN. SAU ĐÓ TÁCH LÀM 2 DÒNG (PICK, REMAIN)

In [156]:
# Các vị trí mà tại đó không cần xuất (marker), chỉ lấy giá trị đầu tiên
df_partial_location = df_removed_before.groupby('SKU').first().reset_index()
df_partial_location = df_partial_location[df_partial_location['SL_con_thieu']!=0]
df_partial_location.head()

,SKU,Order_qty,Stock_qty,UOM,Location,GR_time,SL_luy_ke,SL_con_thieu,Row_num
1,BH209344A1,189,14,PCE,0318A,2020-08-07 13:32:00,197,-8,33
2,BH20A130,157,53,PCE,1611B,2020-08-15 08:28:00,178,-21,10
3,BH20E001,75,9,PCE,0218A,2020-10-28 19:34:00,78,-3,23
4,BY20A871,97,20,PCE,0313B,2020-10-07 14:19:00,99,-2,60


In [157]:
# Tạo dòng số lượng cần pick
df_partial_location_pick = df_partial_location.copy()
df_partial_location_pick['Stock_qty'] = df_partial_location_pick['Stock_qty'] + df_partial_location_pick['SL_con_thieu']

# Trừ row_num đi 0.5 để khi sort thì số lượng cần pick sẽ ở trên
df_partial_location_pick['Row_num']=df_partial_location_pick['Row_num'] - 0.5
df_partial_location_pick.head()

,SKU,Order_qty,Stock_qty,UOM,Location,GR_time,SL_luy_ke,SL_con_thieu,Row_num
1,BH209344A1,189,6,PCE,0318A,2020-08-07 13:32:00,197,-8,32.5
2,BH20A130,157,32,PCE,1611B,2020-08-15 08:28:00,178,-21,9.5
3,BH20E001,75,6,PCE,0218A,2020-10-28 19:34:00,78,-3,22.5
4,BY20A871,97,18,PCE,0313B,2020-10-07 14:19:00,99,-2,59.5


In [158]:
# Tạo dòng số lượng remain (không pick)
df_partial_location_remain = df_partial_location.copy()
df_partial_location_remain['Stock_qty'] = - df_partial_location_remain['SL_con_thieu']
df_partial_location_remain.head()

,SKU,Order_qty,Stock_qty,UOM,Location,GR_time,SL_luy_ke,SL_con_thieu,Row_num
1,BH209344A1,189,8,PCE,0318A,2020-08-07 13:32:00,197,-8,33
2,BH20A130,157,21,PCE,1611B,2020-08-15 08:28:00,178,-21,10
3,BH20E001,75,3,PCE,0218A,2020-10-28 19:34:00,78,-3,23
4,BY20A871,97,2,PCE,0313B,2020-10-07 14:19:00,99,-2,60


In [159]:
# Gộp 2 dòng (pick, remain) lại với nhau
# Ngay sau đó xoá (drop) 2 cột SL_luy_ke và SL_con_thieu
df_location_split = df_partial_location_pick.append(df_partial_location_remain).drop(columns=['SL_luy_ke','SL_con_thieu'])
df_location_split.sort_values('SKU')

,SKU,Order_qty,Stock_qty,UOM,Location,GR_time,Row_num
1,BH209344A1,189,6,PCE,0318A,2020-08-07 13:32:00,32.5
1,BH209344A1,189,8,PCE,0318A,2020-08-07 13:32:00,33.0
2,BH20A130,157,32,PCE,1611B,2020-08-15 08:28:00,9.5
2,BH20A130,157,21,PCE,1611B,2020-08-15 08:28:00,10.0
3,BH20E001,75,6,PCE,0218A,2020-10-28 19:34:00,22.5
3,BH20E001,75,3,PCE,0218A,2020-10-28 19:34:00,23.0
4,BY20A871,97,18,PCE,0313B,2020-10-07 14:19:00,59.5
4,BY20A871,97,2,PCE,0313B,2020-10-07 14:19:00,60.0


### BƯỚC 5: GHÉP 2 DÒNG VỪA TẠO

In [160]:
# Merge (join) 2 file 'inventory.xlsx' và 'order.xlsx' làm 1 dựa trên SKU
df_invt_split = df_order.merge(df_inventory, on='SKU')

In [161]:
# Thêm cột row_num
df_invt_split['Row_num'] = df_invt_split.index

# Tìm những location là partial picking
is_partial = df_invt_split.Row_num.isin(df_partial_location_remain['Row_num'])

# Loại bỏ những location partial picking
df_invt_split = df_invt_split[~is_partial]

# Ghép 2 dòng 
df_invt_split = df_invt_split.append(df_location_split)


In [162]:
# Sort sắp xếp lại cột Row_num
df_invt_split = df_invt_split.sort_values(by=['SKU','GR_time','Row_num'])
df_invt_split.head()

,SKU,Order_qty,Stock_qty,UOM,Location,GR_time,Row_num
0,BH208774,27,11,PCE,0321B,2020-05-30 16:03:00,0.0
1,BH208774,27,9,PCE,1419B,2020-07-16 09:55:00,1.0
2,BH208774,27,3,PCE,0304A,2020-07-16 09:55:00,2.0
3,BH208774,27,4,PCE,0302A,2020-07-22 13:52:00,3.0
4,BH208774,27,2,PCE,0304A,2020-07-24 16:42:00,4.0


In [163]:
# Thêm cột số lượng luỹ kế
df_invt_split["SL_luy_ke"] = df_invt_split.groupby("SKU")["Stock_qty"].apply(lambda x: x.cumsum())
df_invt_split.head()

,SKU,Order_qty,Stock_qty,UOM,Location,GR_time,Row_num,SL_luy_ke
0,BH208774,27,11,PCE,0321B,2020-05-30 16:03:00,0.0,11
1,BH208774,27,9,PCE,1419B,2020-07-16 09:55:00,1.0,20
2,BH208774,27,3,PCE,0304A,2020-07-16 09:55:00,2.0,23
3,BH208774,27,4,PCE,0302A,2020-07-22 13:52:00,3.0,27
4,BH208774,27,2,PCE,0304A,2020-07-24 16:42:00,4.0,29


### BƯỚC 6: LẤY PICKING LIST (INTRUCTION)

In [164]:
# Do tại các vị trí là partial, chúng ta đã tách thành 2 dòng
# Các vị trí được pick khi tại đó 'SL_luy_ke' chưa đạt tới 'Order_qty'
df_pick_instruction = df_invt_split[df_invt_split['Order_qty'] >= df_invt_split['SL_luy_ke']]

In [165]:
# Lấy các cột cần thiết
df_pick_instruction = df_pick_instruction[['SKU','Stock_qty','UOM','Location']]
df_pick_instruction.head()


,SKU,Stock_qty,UOM,Location
0,BH208774,11,PCE,0321B
1,BH208774,9,PCE,1419B
2,BH208774,3,PCE,0304A
3,BH208774,4,PCE,0302A
24,BH209344A1,4,PCE,0222A


In [166]:
# Xuất ra file excel
# Mặc định sẽ xuất chung Folder với file Python
df_pick_instruction.to_excel('python_picking/Picking_List.xlsx', sheet_name='picking', index=False)

### BƯỚC 7: LẤY FILE SỐ LƯỢNG HÀNG CÒN LẠI

In [167]:
# Các vị trí chưa được pick là các vị trí 'SL_luy_ke' vượt 'Order_qty'
df_remain_invt = df_invt_split[df_invt_split['SL_luy_ke'] > df_invt_split['Order_qty']]
df_remain_invt = df_remain_invt[['SKU','Stock_qty','UOM','Location']]
df_remain_invt.head()

,SKU,Stock_qty,UOM,Location
4,BH208774,2,PCE,0304A
5,BH208774,16,PCE,0307A
6,BH208774,4,PCE,0307A
1,BH209344A1,8,PCE,0318A
34,BH209344A1,15,PCE,1217C


In [168]:
# Xuất ra file excel
# Có thể overwrite lên file cũ (tuỳ usecase)
df_remain_invt.to_excel('python_picking/Remain_Inventory.xlsx', sheet_name='remain', index=False)